## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier

def blight_model():
    
    # loading our data to dataframes
    train = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
    address = pd.read_csv('addresses.csv')
    coordinates = pd.read_csv('latlons.csv')
    test = pd.read_csv('test.csv')
    
    # there are some Null values in our target variable column - compliance - that are represent not responsible. 
    # We can drop all rows with compliance = null but I will not do that, because I think we can use some of these rows 
    # in our training set. In order to do that, I especially use payment date and hearing date columns and payment_status 
    # column for verify it. I won't show all off the process here , but for rows I fill as 1.0 - complaint- you can be sure
    # that payment_status = PAID IN FULL.
    
    # let's drop rows- compliance = null - both hearing date and payment date is null because for these row we can not be
    # sure that payment made on time or in 30 days after hearing date.
    del_r = train[(train['hearing_date'].isnull()) & (train['payment_date'].isnull()) & (train['compliance'].isnull())].index.tolist()
    train.drop(del_r, inplace = True)
    
    # if hearing date is not null and payment date is null we can conclude that there is no payment so lets fill these rows
    # with 0.0, non-compliant.
    train['compliance'][(train['hearing_date'].isnull() == False) & (train['payment_date'].isnull()) 
      & (train['compliance'].isnull())] = train['compliance'][(train['hearing_date'].isnull() == False) & (train['payment_date'].isnull()) 
      & (train['compliance'].isnull())].fillna(0.0)
    
    # convert ticket_issued_date, hearing_date and payment_date features' data types to datetime in order to 
    # make operations on them
    train['ticket_issued_date'] = train['ticket_issued_date'].apply(pd.to_datetime)
    train['hearing_date'] = train['hearing_date'].apply(pd.to_datetime)
    train['payment_date'] = train['payment_date'].apply(pd.to_datetime)
    
    def take_time_difference_ph(row):
        # this function take time difference between payment_date and hearing_date in terms of days
        if pd.isnull(row['hearing_date']) or pd.isnull(row['payment_date']):
            return None
        return (row['payment_date'] - row['hearing_date']).days
    
    # creating new feature - ph_diff - for train set
    train['ph_diff'] = train.apply(take_time_difference_ph, axis = 1)
    
    # for rows for hearing date is null but payment date is not null, let's fill these rows with 1.0, compliant.
    # this decision maybe not seem very right to you but I prefer to do that.
    train['compliance'][(train['hearing_date'].isnull()) & (train['payment_date'].isnull() == False) 
      & (train['compliance'].isnull())] = train['compliance'][(train['hearing_date'].isnull()) & (train['payment_date'].isnull() == False) 
      & (train['compliance'].isnull())].fillna(1.0)
    
    # for rows for hearing date is not null and also payment date is not null, I use ph_diff variable to make this decision.
    # if ph_diff is lower than 30, it is okay to fill with 1.0- compliant, else fill with 0.0, non-compliant.
    train['compliance'][(train['ph_diff'] <= 30)
      & (train['compliance'].isnull())] = train['compliance'][(train['ph_diff'] <= 30)
      & (train['compliance'].isnull())].fillna(1.0)
    
    train['compliance'].fillna(0.0, inplace = True)
    
    # train_y is our target varible
    train_y = train['compliance']
    
    # drop all columns that included only train data
    train.drop(['payment_amount', 'payment_date', 'payment_status','balance_due', 'collection_status', 'compliance_detail', 
            'compliance', 'ph_diff'], axis = 1, inplace = True)
    
    def take_time_difference(row):
        # this function take time difference between hearing_date and ticket_issued_date in terms of days
        if pd.isnull(row['hearing_date']):
            return None
        return (row['hearing_date'] - row['ticket_issued_date']).days
    
    # creating new feature - ht_diff - for train
    train['ht_diff'] = train.apply(take_time_difference, axis = 1)
    
    # drop columns that will not used for our model from both train and test data
    train.drop(['violator_name', 'violation_zip_code', 'non_us_str_code', 'country', 'grafitti_status', 'admin_fee', 'state_fee'],
          axis = 1, inplace = True)
    test.drop(['violator_name', 'violation_zip_code', 'non_us_str_code', 'country', 'grafitti_status', 'admin_fee', 'state_fee'],
          axis = 1, inplace = True)
    
    # convert ticket_issued_data and hearing_date columns' data types to datetime and create ht_diff featre for test data
    test['ticket_issued_date'] = test['ticket_issued_date'].apply(pd.to_datetime)
    test['hearing_date'] = test['hearing_date'].apply(pd.to_datetime)
    test['ht_diff'] = test.apply(take_time_difference, axis = 1)
    
     # fill NaN values in ht_diff with median
    test['ht_diff'].fillna(test['ht_diff'].median(), inplace = True)
    train['ht_diff'].fillna(train['ht_diff'].median(), inplace = True)
    
    train['fine_amount'].fillna(train['fine_amount'].median(), inplace = True)
    
    # drop columns that will not used for our model from both train and test data
    train.drop(['inspector_name', 'violation_street_number', 'violation_street_name', 'mailing_address_str_number', 
            'mailing_address_str_name', 'ticket_issued_date', 'hearing_date', 'violation_description'],
          axis = 1, inplace = True)
    test.drop(['inspector_name', 'violation_street_number', 'violation_street_name', 'mailing_address_str_number', 
            'mailing_address_str_name', 'ticket_issued_date', 'hearing_date', 'violation_description'],
          axis = 1, inplace = True)
    
    # fill None in the state column with most occuring state - MI
    train['state'].fillna('MI', inplace = True)
    test['state'].fillna('MI', inplace = True)
    
    # keep only most occuring 20 state and replace others with the value X
    test['state'].replace(train['state'].value_counts().index[20:], 'X', inplace = True)
    train['state'].replace(train['state'].value_counts().index[20:], 'X', inplace = True)
    
    # keep only most occuring 11 violation code and replace others with value X_viocode
    test.ix[(test['violation_code'].isin(train['violation_code'].value_counts().index[:11])==False), 'violation_code'] = 'X_viocode'
    train.ix[(train['violation_code'].isin(train['violation_code'].value_counts().index[:11])==False), 'violation_code'] = 'X_viocode'
    
    # fill None in the zip code column with most occuring zip code - it is 48221 for train and 48235 for test
    train['zip_code'].fillna(48221, inplace = True)
    test['zip_code'].fillna(48235, inplace = True)
    
    # keep only most occuring 11 violation code and replace others with value X_zipcode
    test.ix[(test['zip_code'].isin(train['zip_code'].value_counts().index[:11])==False), 'zip_code'] = 'X_zipcode'
    train.ix[(train['zip_code'].isin(train['zip_code'].value_counts().index[:11])==False), 'zip_code'] = 'X_zipcode'
    
    # In order to identify unique city names easily, convert all of them to lowercase
    train['city'] = train['city'].str.lower()
    test['city'] = test['city'].str.lower()
    
    # replace some short version of city names with normal for most occuring city names
    test['city'].replace(['det', 'det.'], 'detroit', inplace = True)
    train['city'].replace(['det', 'det.'], 'detroit', inplace = True)
    
    test['city'].replace(['w. bloomfield'], 'west bloomfield', inplace = True)
    train['city'].replace(['w. bloomfield'], 'west bloomfield', inplace = True)
    
    # keep only most occuring 7 city and replace others with value X_city
    test.ix[(test['city'].isin(train['city'].value_counts().index[:7])==False), 'city'] = 'X_city'
    train.ix[(train['city'].isin(train['city'].value_counts().index[:7])==False), 'city'] = 'X_city'
    
    # these are the possible values of disposition for test data. 
    # select these values only and replace other values with X_disposition
    selected_dis = ['Responsible by Default', 'Responsible by Admission', 'Responsible by Determination']
    test.ix[(test['disposition'].isin(selected_dis)==False), 'disposition'] = 'X_disposition'
    train.ix[(train['disposition'].isin(selected_dis)==False), 'disposition'] = 'X_disposition'
    
    # set address column as index 
    coordinates.index = coordinates['address']
    coordinates.drop('address', axis=1, inplace=True)
    
    # join address and coordinates dataframes based on their common column address
    # actually we want to take lat and lon features and this process map ticket id to lat and lon
    x = address.join(coordinates, on = 'address', how = 'left')
    x.drop(['address'], axis = 1, inplace = True)
    
    # there are few null values, lets fll them with mean values
    x['lat'].fillna(x['lat'].mean(), inplace = True)
    x['lon'].fillna(x['lon'].mean(), inplace = True)
    
    # join lat and lon features to our train and test data
    train = train.join(x.set_index('ticket_id'), on = 'ticket_id', how = 'left')
    test = test.join(x.set_index('ticket_id'), on = 'ticket_id', how = 'left')
    
    # for our categorical features, let's get dummies and drop actual columns
    train = pd.concat([train.drop(['agency_name', 'city', 'state', 'zip_code', 'violation_code', 'disposition'], axis=1), 
                   pd.get_dummies(train[['agency_name', 'city', 'state', 'zip_code', 'violation_code', 'disposition']])],axis=1)
    test = pd.concat([test.drop(['agency_name', 'city', 'state', 'zip_code', 'violation_code', 'disposition'], axis=1), 
                  pd.get_dummies(test[['agency_name', 'city', 'state', 'zip_code', 'violation_code', 'disposition']])], axis=1)
    
    # set ticket_id column as index for both train and test data. Actually this is necessary for test
    # because of our return format but not for train.
    train.set_index('ticket_id', inplace = True)
    test.set_index('ticket_id', inplace = True)
    
    # these agency names are only in train data but not in test data so let's drop them 
    train.drop(['agency_name_Health Department', 'agency_name_Neighborhood City Halls'], axis=1, inplace = True)
    
    # let's make sure there are no duplicated columns in train set.
    train = train.loc[:,~train.columns.duplicated()]
    
    # let's make sure that the ordering of features same both in train and test set
    test = test[list(train.columns)]
    
    # these are the names of the features that we use in our final model.
    # I choose them according to the feature importances 
    selected_features = ['violation_code_9-1-36(a)', 'disposition_X_disposition', 'ht_diff','disposition_Responsible by Default',
     'judgment_amount', 'fine_amount', 'disposition_Responsible by Determination', 'late_fee', 'disposition_Responsible by Admission',
     'discount_amount','lon', 'violation_code_9-1-103(C)', 'lat', 'agency_name_Department of Public Works',
                         'agency_name_Buildings, Safety Engineering & Env Department', 'violation_code_9-1-81(a)']
    
    # let's use random forest classifier in order to train our model
    clf = RandomForestClassifier(n_estimators = 300, max_features = 0.33, max_depth =4, random_state = 48)
    clf.fit(train[selected_features], train_y)
    
    # because our output data type float32, let's make sure that is float32.
    return pd.Series(np.float32(clf.predict_proba(test[selected_features])[:, 1]), index = test.index)

In [2]:
import warnings
warnings.filterwarnings('ignore')

blight_model()

ticket_id
284932    0.047677
285362    0.036120
285361    0.047677
285338    0.048268
285346    0.048268
285345    0.048268
285347    0.048268
285342    0.500807
285530    0.036836
284989    0.036556
285344    0.048268
285343    0.036836
285340    0.036836
285341    0.048268
285349    0.048268
285348    0.048268
284991    0.036556
285532    0.036836
285406    0.036156
285001    0.036726
285006    0.035956
285405    0.036120
285337    0.036156
285496    0.047677
285497    0.047677
285378    0.036120
285589    0.036156
285585    0.047677
285501    0.047677
285581    0.036120
            ...   
376367    0.035156
376366    0.036356
376362    0.195064
376363    0.234266
376365    0.035156
376364    0.036356
376228    0.036523
376265    0.036356
376286    0.201707
376320    0.036356
376314    0.036326
376327    0.236567
376385    0.238562
376435    0.470249
376370    0.556125
376434    0.047732
376459    0.047732
376478    0.036052
376473    0.036326
376484    0.035369
376482    0.035193
37